# NYSK Corpus Preprocessing

Author: Paul Sheridan

Goal: Apply some basic text analysis preprocessing steps to the NYSK coupus documents and output the result in JSON format.

Input file: nysk.xml (Available for download at https://archive-beta.ics.uci.edu/dataset/260/nysk)

Output file: nysk-preprocessed.json

## Load modules

In [14]:
import re, string, unicodedata
import nltk
import contractions
import inflect
import json
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

## Define custom NLP functions

In [17]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    """Stem and lemmmatize words in one go"""
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

def normalize(words):
    """Run selected above functions as a one-liner"""
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

## Read NYSK XML file and store its contents in a list

In [15]:
xml_file_path = 'nysk.xml' # write local path to nysk.xml file file here
infile = open(xml_file_path, 'r')
contents = infile.read()
soup = BeautifulSoup(contents, 'xml')
body_texts = soup.find_all('text')
N = len(soup.find_all('title'))
print('N = {0}'.format(N))

N = 10421


## Apply NLP preprocessing steps to corpus docs

In [20]:
corpus = []
for text in body_texts:
    sample = text.get_text()
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    stems, lemmas = stem_and_lemmatize(words)
    corpus.append(lemmas)

## Write preprocessed NYSK corpus data to JSON file

In [21]:
json_file_path = 'nysk-preprocessed.json' # write output nysk-preprocessed.json file path here
with open(json_file_path, 'w') as outfile:
    json.dump(corpus, outfile)